In [0]:
#importing the libraries:
import pandas as pd
import numpy as np

In [0]:
#importing the dataset:
df = pd.read_csv('mobile.csv')

In [0]:
# from pandas_profiling import ProfileReport
# prof = ProfileReport(df)
# prof.to_file(output_file='output.html')

In [0]:
df.sample(5)

In [0]:
df.info()

In [0]:
df.isnull().sum()

In [0]:
#dropping out cols that are not required to predict the price:
df.drop(columns=['url','product_id','listing_id','availability','currency'],inplace=True)

In [0]:
df.sample(5)

In [0]:
df['brand'].value_counts()

In [0]:
#working on 'title' column:
# fetching out mobile models:
df['model'] = df['title'].str.split("(",n=1,expand=True)[0]

In [0]:
df

In [0]:
#fetching out colors:
df['color'] = df['title'].str.split("(",n=1,expand=True)[1].str.split(",",n=1,expand=True)[0]

In [0]:
df.info()

In [0]:
df['ram'] = df['ram'].str.split(" ",n=1,expand=True)[0]

In [0]:
#filling out missing values in ram:
df['ram'].fillna(df['ram'].mode()[0],inplace=True)

In [0]:
#converting datatype of ram into int:
df['ram'] = df['ram'].astype(int)

In [0]:
df.info()

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
sns.distplot(np.log(df['original_price']))

In [0]:
sns.barplot(x=df['ram'], y=df['original_price'])
plt.xticks(rotation='vertical')
plt.show

In [0]:
df['brand'].value_counts().plot(kind='bar')

In [0]:
sns.barplot(x=df['brand'], y=df['original_price'])
plt.xticks(rotation='vertical')
plt.show

In [0]:
#extracting other features:
#rom of each smartphone:
temp_df = df['title'].str.split(",",n=2,expand=True)[1]

In [0]:
temp_df = temp_df.str.replace("TB","000")
temp_df = temp_df.str.replace("GB","")
temp_df = temp_df.str.replace(" ","")

In [0]:
temp_df.value_counts()

In [0]:
df['rom'] = temp_df.str.split(")",n=2,expand=True)[0]

In [0]:
df['rom'].value_counts()

In [0]:
df['rom'].isnull().sum()

In [0]:
#fetching the index number:
df[df['rom']=='ChocolateBrown'].index

In [0]:
df['title'][688]

In [0]:
#replacing chocolatebrown to the mode of ram:
df['rom'] = df['rom'].str.replace("ChocolateBrown",'8')

In [0]:
df['rom'].value_counts()

In [0]:
#converting to int:
df['rom'] = df['rom'].astype(int)

In [0]:
df.info()

In [0]:
sns.barplot(x=df['rom'], y=df['original_price'])
plt.xticks(rotation='vertical')
plt.show

In [0]:
df['highlights'][55]

In [0]:
temp_df1 = df['highlights'].str.split("ROM",n=2,expand=True)

In [0]:
temp_df1[1]

In [0]:
#fetching out expandable memory:
df['expandable'] = temp_df1[1].str.replace(" TB","000 GB")

In [0]:
df['expandable'] = df['expandable'].str.split("GB",n=2,expand=True)[0]

In [0]:
df['expandable'] = df['expandable'].str.split(" ",n=4,expand=True)[4]

In [0]:
df['expandable'].isnull().sum()

In [0]:
df['expandable'] = df['expandable'].str.split(")",n=2,expand=True)[0]

In [0]:
df['expandable'] = df['expandable'].str.replace("inch",'0')

In [0]:
df['expandable'] = df['expandable'].astype(int)

In [0]:
df['expandable'].value_counts()

In [0]:
df['highlights'][5]

In [0]:
#fetching out size of the phone:
df['size(inch)'] = df['highlights'].str.split("(",n=2,expand=True)[1]

In [0]:
df['size(inch)'] = df['size(inch)'].str.split(" ",n=3,expand=True)[0]

In [0]:
df['size(inch)'].isnull().sum()

In [0]:
df['size(inch)'] = df['size(inch)'].astype(float)

In [0]:
df['size(inch)'].value_counts()

In [0]:
df['highlights'][166]

In [0]:
#fetching out rear and front camera details:
camera = df['highlights'].str.split("Display",n=2,expand=True)[1]

In [0]:
camera

In [0]:
df['camera'] = camera.str.split("|",n=2,expand=True)[0]

In [0]:
rear = df['camera'].str.split(" ",n=4,expand=True)

In [0]:
df['rear_camera'] = rear[1].str.split("MP",n=2,expand=True)[0]

In [0]:
df['rear_camera'].value_counts()

In [0]:
df['rear_camera'] = df['rear_camera'].astype(int)

In [0]:
df['rear_camera'].isnull().sum()

In [0]:
front = camera.str.split("|",n=2,expand=True)[1]

In [0]:
df['front_camera'] = front.str.split(" ",n=2,expand=True)[1]

In [0]:
df['front_camera'] = df['front_camera'].str.replace("MP","")

In [0]:
df['front_camera'].value_counts()

In [0]:
df['front_camera'].isnull().sum()

In [0]:
df['front_camera'].fillna(0,inplace=True)

In [0]:
df['front_camera'] = df['front_camera'].astype(int)

In [0]:
df.info()

In [0]:
#sorting required cols:
#new_df = df[['brand','model','color','ram','rom','expandable','rear_camera','front_camera','size(inch)','original_price']]
new_df = df[['brand','model','color','ram','rom','expandable','rear_camera','front_camera','size(inch)','one_stars_count','two_stars_count','three_stars_count','four_stars_count','five_stars_count','original_price']]

In [0]:
new_df

In [0]:
new_df.corr()['original_price']

In [0]:
sns.heatmap(new_df.corr())

In [0]:
X = new_df.iloc[:,:-1]
y = np.log(new_df['original_price'])

In [0]:
X

In [0]:
y

## Building the model

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [0]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.model_selection import cross_val_score

## Linear Regression

In [0]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf1',OneHotEncoder(sparse=False,drop='first',handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

step2 = LinearRegression()

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2Score: ',r2_score(y_test,y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

## Ridge

In [0]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf1',OneHotEncoder(sparse=False,drop='first',handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

step2 = Ridge(alpha=5)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2Score: ',r2_score(y_test,y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

## Lasso

In [0]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf1',OneHotEncoder(sparse=False,drop='first',handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

step2 = Lasso(alpha=0.0001)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2Score: ',r2_score(y_test,y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

## KNN

In [0]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf1',OneHotEncoder(sparse=False,drop='first',handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

step2 = KNeighborsRegressor(n_neighbors=1)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2Score: ',r2_score(y_test,y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

## Decision Tree

In [0]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf1',OneHotEncoder(sparse=False,drop='first',handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

step2 = DecisionTreeRegressor(max_depth=230)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2Score: ',r2_score(y_test,y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

## Random Forest

In [0]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf1',OneHotEncoder(sparse=False,drop='first',handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

step2 = RandomForestRegressor(n_estimators=100,random_state=5, max_samples=0.5, max_features=0.80, max_depth=30)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2Score: ',r2_score(y_test,y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

## Gradient Boosting Regressor

In [0]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf1',OneHotEncoder(sparse=False,drop='first',handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

step2 = GradientBoostingRegressor(n_estimators=300)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2Score: ',r2_score(y_test,y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

## Adaboost Regressor

In [0]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf1',OneHotEncoder(sparse=False,drop='first',handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

step2 = AdaBoostRegressor(learning_rate=0.05, n_estimators=200)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2Score: ',r2_score(y_test,y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

## SVR

In [0]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf1',OneHotEncoder(sparse=False,drop='first',handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

step2 = SVR(C=10000, epsilon=0.1)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2Score: ',r2_score(y_test,y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

## XGB

In [0]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf1',OneHotEncoder(sparse=False,drop='first',handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

step2 = XGBRegressor(n_estimators=200, learning_rate=0.15, max_depth=15)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2Score: ',r2_score(y_test,y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

## Exporting the model

In [0]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))
pickle.dump(new_df,open('df.pkl','wb'))